## Plan for running experiments

* Debug
    * That thing with autoencoders?
    * Check random_state (setting for models? Setting for dataset?)
* Prepare for launch
    * Make sure this notebook runs fine
    * Run this notebook, have analysis to verify
* Launch EC2 instances (instance type: XXX, Anaconda's AMI)
    * Upgrade Dask
    * Install PyTorch/Skorch
    * Upgrade jupyter notebooks?
    * Run Dask workers on each machine
* Run

## Setup

In [1]:
# %matplotlib inline
# %load_ext autoreload
# %autoreload 2

In [2]:
import distributed
from distributed import Client, LocalCluster
#cluster = LocalCluster(n_workers=16)
#client = Client(cluster)
client = Client("localhost:8786")
client

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 20 Cores: 20 Memory: 40.00 GB


In [3]:
# import subprocess
# def debug_loop():
#     subprocess.call("pip install git+https://github.com/stsievert/dask-ml@hyperband-scale".split(" "))
#     import dask_ml
#     return dask_ml.__version__

In [4]:
# %time debug_loop()

In [5]:
# client.run(debug_loop)

In [6]:
# %time client.restart()

In [7]:
%time client.upload_file('autoencoder.py')

CPU times: user 11.1 ms, sys: 1.54 ms, total: 12.7 ms
Wall time: 4.22 s


In [8]:
import dask_ml
from dask_ml.model_selection import HyperbandSearchCV
import dask_ml
dask_ml.__version__

'0.4.2.dev455+gd68fe46'

In [9]:
dask_ml.__file__

'/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/dask_ml/__init__.py'

## Data
See below for an image.

In [10]:
# %load_ext autoreload
# %autoreload 2

In [11]:
import noisy_mnist

Xs = []
ys = []
for random_state in range(4):
    _X, _y = noisy_mnist.dataset(random_state=random_state)
    print(_X.mean())
    Xs.append(_X)
    ys.append(_y)


0.1865414
0.19537193
0.19534668
0.1865923


In [12]:
import numpy as np
# _X = np.concatenate([_X[:-3] for _X in Xs])
# _y = np.concatenate([_y[:-3] for _y in ys])

_X = np.concatenate(Xs)
_y = np.concatenate(ys)
# _X = _X[:-8]
# _y = _y[:-8]

_X.shape, _X.dtype, _X.min(), _X.max()

((280000, 784), dtype('float32'), 0.0, 1.0)

In [13]:
_y.shape, _y.dtype, _y.min(), _y.max()

((280000, 784), dtype('float32'), 0.0, 1.0)

In [14]:
import dask.array as da
n, d = _X.shape
chunks = n // 8
print(chunks)
X = da.from_array(_X, chunks=(chunks, d))
y = da.from_array(_y, chunks=chunks)
X, y

35000


(dask.array<array, shape=(280000, 784), dtype=float32, chunksize=(35000, 784)>,
 dask.array<array, shape=(280000, 784), dtype=float32, chunksize=(35000, 784)>)

In [15]:
X.chunks

((35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000), (784,))

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import check_random_state

# rng = check_random_state(42)
# cols = 8
# w = 1.0
# fig, axs = plt.subplots(figsize=(cols*w, 2*w), ncols=cols, nrows=2)

# show_X = _X
# show_y = _y
# for col, (upper, lower) in enumerate(zip(axs[0], axs[1])):
#     if col == 0:
#         upper.text(-28, 14, 'ground\ntruth')
#         lower.text(-28, 14, 'input')
#     i = rng.choice(len(X))
#     noisy = show_X[i].reshape(28, 28)
#     clean = show_y[i].reshape(28, 28)
#     kwargs = {'cbar': False, 'xticklabels': False, 'yticklabels': False, 'cmap': 'gray_r'}
#     sns.heatmap(noisy, ax=lower, **kwargs)
#     sns.heatmap(clean, ax=upper, **kwargs)
# plt.savefig("imgs/input-output.svg", bbox_inches="tight")
# plt.show()

<Figure size 800x200 with 16 Axes>

## Model

I use a deep learning library (PyTorch) for this model, at least through the scikit-learn interface for PyTorch, [skorch].

[skorch]:https://github.com/dnouri/skorch

In [ ]:
from autoencoder import Autoencoder, NegLossScore
import torch
# from sklearn.model_selection import ParameterSampler
import torch

def trim_params(**kwargs):
    if kwargs['optimizer'] != 'Adam':
        kwargs.pop('optimizer__amsgrad', None)
    if kwargs['optimizer'] == 'Adam':
        kwargs.pop('optimizer__lr', None)
    if kwargs['optimizer'] != 'SGD':
        kwargs.pop('optimizer__nesterov', None)
        kwargs.pop('optimizer__momentum', None)
    kwargs['optimizer'] = getattr(torch.optim, kwargs['optimizer'])
    return kwargs

class TrimParams(NegLossScore):
    def set_params(self, **kwargs):
        kwargs = trim_params(**kwargs)
        return super().set_params(**kwargs)

model = TrimParams(
    module=Autoencoder,
    criterion=torch.nn.BCELoss,
    warm_start=True,
    train_split=None,
    max_epochs=1,
    callbacks=[]
)

I don't show it here; I'd rather concentrate on tuning hyperparameters. But briefly, it's a simple fully connected 3 hidden layer autoencoder with a latent dimension of 49.

## Parameters

The parameters I am interested in tuning are

* model
    * initialization
    * activation function
    * weight decay (which is similar to $\ell_2$ regularization)
* optimizer
    * which optimizer to use (e.g., Adam, SGD)
    * batch size used to approximate gradient
    * learning rate (but not for Adam)
    * momentum for SGD
    
After looking at the results, I think I was too exploratory in my tuning of step size. I should have experimented with it more to determine a reasonable range.

In [33]:
import numpy as np

params = {
    'module__init': ['xavier_uniform_',
                     'xavier_normal_',
                     'kaiming_uniform_',
                     'kaiming_normal_',
                    ],
    'module__activation': ['ReLU', 'LeakyReLU', 'ELU', 'PReLU'],
    'optimizer': ["SGD"] * 5 + ["Adam"] * 2,
    'batch_size': [32, 64, 128, 256, 512],
    'optimizer__lr': np.logspace(1, -1.5, num=1000),
    'optimizer__weight_decay': [0]*200 + np.logspace(-5, -3, num=1000).tolist(),
    'optimizer__nesterov': [True],
    'optimizer__momentum': np.linspace(0, 1, num=1000),
    'train_split': [None],
}

I am testing `optimizer` to be `SGD` or `Adam` to test "[The Marginal Value of Adaptive Gradient Methods in Machine Learning][marginal]". From their abstract,

> We observe that the solutions found by adaptive methods generalize worse (often sig- nificantly worse) than SGD, even when these solutions have better training performance. These results suggest that practitioners should reconsider the use of adaptive methods to train neural networks.

Their experiments in Figure 1b show that non-adaptive methods (SGD and heavy ball) perform much better than adaptive methods.

They have to do some tuning for this. **Can we replicate their result?**

[marginal]:https://arxiv.org/pdf/1705.08292.pdf

In [34]:
# for debugging; ignore this cell

# from sklearn.linear_model import SGDClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import ParameterSampler
# import dask.array as da
# import numpy as np
# model = SGDClassifier()
# params = {'alpha': np.logspace(-7, 0, num=int(1e6))}

# n, d = int(10e3), 784
# _X, _y = make_classification(n_samples=n, n_features=d,
#                              random_state=1)
# X = da.from_array(_X, chunks=(n // 10, d))
# y = da.from_array(_y, chunks=n // 10)
# X, y

In [35]:
import json
import msgpack
import pandas as pd

def fmt(obj):
    if isinstance(obj, list):
        return [fmt(v) for v in obj]
    if isinstance(obj, dict):
        return {k: fmt(v) for k, v in obj.items()}
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

import msgpack
from sklearn.externals import joblib

def save_search(search, today, prefix, X, y, task_stream):
    pre = f"{today}-{prefix}-"
    print("    " + prefix, search.best_score_)
    with open(pre + "test.npz", "wb") as f:
        y_hat = search.predict(X)
        np.savez(f, X=X, y=y, y_hat=y_hat)
    # skorch models aren't pickable
    with open(pre + "params.json", "w") as f:
        params = {k: fmt(v) for k, v in search.get_params().items() if "estimator" not in k and "param_distribution" not in k}
        json.dump(params, f)
    # with open(pre + "best-model.joblib", "wb") as f:
    #     joblib.dump(search.best_estimator_, f)
    with open(pre + "best-params-and-score.json", "w") as f:
        json.dump({"params": search.best_params_, "score": search.best_score_}, f)

    with open(pre + "history.json", 'w') as f:
        json.dump(search.history_, f)

    with open(pre + "cv_results.json", 'w') as f:
        json.dump(fmt(search.cv_results_), f)
     
    timing_stats = client.profile(filename=pre + f"profile-graph.html")
    with open(pre + f"timing.json", "w") as f:
        json.dump(timing_stats[0], f)
        
    pd.DataFrame(task_stream.data).to_msgpack(pre + "task-stream.msgpack")

In [36]:
today = "2019-06-25/o-v3"
# today = "_debug/"

## Hyperparameter optimization

In [37]:
X

dask.array<array, shape=(280000, 784), dtype=float32, chunksize=(35000, 784)>

In [38]:
X.chunks

((35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000), (784,))

In [39]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_test

(dask.array<concatenate, shape=(224000, 784), dtype=float32, chunksize=(28000, 784)>,
 dask.array<concatenate, shape=(56000, 784), dtype=float32, chunksize=(7000, 784)>)

In [40]:
X_train

dask.array<concatenate, shape=(224000, 784), dtype=float32, chunksize=(28000, 784)>

In [41]:
from sklearn.linear_model import SGDClassifier

max_iter = 243
history = {}
cv_results = {}
searches = {}

In [42]:
from dask_ml.model_selection import HyperbandSearchCV

fit_params = {}
if isinstance(model, SGDClassifier):
    fit_params = {'classes': da.unique(y).compute()}
   

### Hyperband

In [43]:
from dask_ml.model_selection import IncrementalSearchCV
from time import time
from distributed import get_task_stream
import warnings

In [44]:
def run_comparison(model, params, max_iter, client, fit_params=None, random_state=None):
    if fit_params is None:
        fit_params = {}
    assert isinstance(random_state, int)
    
#   print(f"    patience {random_state} done in {time() - start:0.2f}")
    print("starting hyperband")
    start = time()
    # Hyperband
    hyperband = HyperbandSearchCV(
        model,
        params,
        max_iter=max_iter,
        random_state=random_state,
        test_size=0.2,
    )
    with get_task_stream() as ts:
        hyperband.fit(X_train, y_train, **fit_params)
        
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        save_search(hyperband, today, f"hyperband-{random_state}", X_test.compute(), y_test.compute(), ts)
    print(f"    hyperband {random_state} done in {time() - start:0.2f}")

    print("    starting hyperband+sop")
    start = time()
    h_sop = HyperbandSearchCV(
        model,
        params,
        max_iter=max_iter,
        random_state=random_state,
        patience=True,
        test_size=0.2,
    )
    with get_task_stream() as ts:
        h_sop.fit(X_train, y_train, **fit_params)
        
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        save_search(h_sop, today, f"hyperband-w-patience-{random_state}", X_test.compute(), y_test.compute(), ts)
    print(f"    hyperband+sop {random_state} done in {time() - start:0.2f}")

    total_calls = hyperband.metadata['partial_fit_calls']
    num_calls = max_iter

    n_workers = 20 or len(client.cluster.workers)
    num_models = max(n_workers, total_calls // num_calls)
    assert num_calls == 243
    assert num_models == 20
    assert hyperband.metadata["partial_fit_calls"] == 4743

    print("    starting patience")
    num_calls = 243
    num_models = 20
    start = time()
    patience = IncrementalSearchCV(
        model,
        params,
        decay_rate=0,
        patience=False,
        n_initial_parameters=num_models,
        max_iter=num_calls,
        random_state=random_state,
        scores_per_fit=10,
        test_size=0.2,
    )
    with get_task_stream() as ts:
        patience.fit(X_train, y_train, **fit_params)
        
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        save_search(patience, today, f"patience-{random_state}", X_test.compute(), y_test.compute(), ts)


    return None #hyperband, h_sop, passive, patience

In [45]:
patience = 243 // 10
patience

24

In [46]:
# from sklearn.base import BaseEstimator
# from time import sleep
# class LinearFunction(BaseEstimator):
#     def __init__(self, intercept=0, slope=1, **kwargs):
#         self.intercept = intercept
#         self.slope = slope
#         self._partial_fit_calls = 0
        
#     def fit(self, X, y):
#         return self
#     def partial_fit(self, X, y):
#         self._partial_fit_calls += 1
#         sleep(0.05)
#         return self
    
#     def score(self, X, y):
#         return self.slope * self._partial_fit_calls + self.intercept
#     def predict(self, X):
#         return self.slope * self._partial_fit_calls + self.intercept
        
# model = LinearFunction()
# params = {"slope": np.linspace(0, 0.0005 / 25, num=1000)}

In [47]:
N0 = 150
N = 100

for random_state in range(N0, N0 + N):
    start = time()
    _ = run_comparison(model, params, max_iter, client, random_state=random_state, fit_params=fit_params)
    print(f"comparison {random_state} done in {time() - start:0.2f}")

starting hyperband


tornado.application - ERROR - Multiple exceptions in yield list
Traceback (most recent call last):
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 883, in callback
    result_list.append(f.result())
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/dask_ml/model_selection/_incremental.py", line 578, in _fit
    random_state=self.random_state,
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/mnt/ws/home/ssievert/anaconda3/lib/python3.7/site-packages/dask_ml/model_selection/_incremental.py", line 208, in _fit
    metas = yield client.gather(new_scores)
  File

KeyError: 'startstops'

In [ ]:
# started patience at 1:57

In [44]:
timing_stats = client.profile()

with open("final-timings.json", "w") as f:
    json.dump(timing_stats, f)

data, fig = client.get_task_stream(plot=True)

len(data)

In [51]:
fig

Figure(id='bk-task-stream-plot', ...)

In [58]:
import pandas as pd
df = pd.DataFrame(list(data))

In [59]:
df.head()

,key,nbytes,startstops,status,thread,type,worker
0,dict-583ab4d6-cc20-4bde-951a-c2854be3b2ce,240,"((compute, 1553435505.039325, 1553435505.03934...",OK,140036838618880,b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00...,tcp://172.31.25.234:45003
1,dict-fb5ab1f4-d792-446d-b9a6-1eea3a78f379,240,"((compute, 1553435505.0397773, 1553435505.0397...",OK,139998560577280,b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00...,tcp://172.31.18.35:46685
2,_create_model-d4908a4e892a1a4c21ea76b6015127a1,360,"((compute, 1553435505.0285432, 1553435505.0482...",OK,140393498208000,b'\x80\x04\x95\x16\x00\x00\x00\x00\x00\x00\x00...,tcp://172.31.22.106:41967
3,_create_model-ad3285c9b623bb50ebefa67632be0e83,360,"((compute, 1553435505.02852, 1553435505.048227...",OK,140393498470144,b'\x80\x04\x95\x16\x00\x00\x00\x00\x00\x00\x00...,tcp://172.31.22.106:40967
4,_create_model-f426d78a1fbbc129561420a60c0160a5,360,"((compute, 1553435505.027857, 1553435505.04965...",OK,140393498732288,b'\x80\x04\x95\x16\x00\x00\x00\x00\x00\x00\x00...,tcp://172.31.22.106:35363


In [60]:
df.to_msgpack("times.msgpack")

In [53]:
type(data)

tuple

In [54]:
len(data)

42609